In [ ]:
import torch
from transformers import AutoFeatureExtractor, WhisperForAudioClassification
from datasets import load_dataset


In [33]:
import torch as t
import numpy as np
t.cuda.set_device(8)
audio_dir = "/raid/amana/lavish_multi_model/emotion_detection/data/raw_audio/"

In [48]:
import numpy as np 
import soundfile as sf
import librosa

def getVggoud_proc(filename):
    audio_length = 1.5
    idx=0
    samples, samplerate = librosa.load(filename)

    if samples.shape[0] > 16000*(audio_length+0.1):
        sample_indx = np.linspace(0, samples.shape[0]-16000*(audio_length+0.1), num=1, dtype=int)
        samples = samples[sample_indx[idx]:sample_indx[idx]+int(16000*audio_length)]
    else:
        samples = np.tile(samples,int(audio_length))[:int(16000*audio_length)]

    samples[samples > 1.] = 1
    samples[samples < -1.] = -1
    
    return samples

def resample_audio(arr,sr):
    return librosa.util.fix_length(arr, size=sr)

In [49]:
from os import listdir
from os.path import isfile, join
import pandas as pd

def get_dataset(audio_dir):
    data = pd.read_csv("/raid/amana/lavish_multi_model/emotion_detection/data/text_data.csv")
    data['filename'] = [f'dia{a}_utt{b}' for a,b in zip(data['Dialogue_ID'],data['Utterance_ID'])]
    labels = list(data['Emotion'])
    onlyfiles = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f))]
    res = []
    shapes = []
    for a,label in zip(onlyfiles,labels):
        res1 = getVggoud_proc(audio_dir+"/"+a)
        shapes.append(res1.shape)
        res.append([res1,label])
    shapes = np.array(shapes)
    true_shapes = shapes>5000
    res = [a for a,b in zip(res,true_shapes) if b[0] == True]
    final_len = int(np.mean(shapes))
    final_dataset = [[resample_audio(a[0],final_len),a[1]] for a in res]
    return final_dataset


def get_dataset_2(audio_dir):
    data = pd.read_csv("/raid/amana/lavish_multi_model/emotion_detection/data/text_data.csv")
    data['filename'] = [f'dia{a}_utt{b}' for a,b in zip(data['Dialogue_ID'],data['Utterance_ID'])]
    labels = list(data['Emotion'])
    onlyfiles = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f))]
    res = []
    shapes = []
    for a,label in zip(onlyfiles,labels):
        res1 = getVggoud_proc(audio_dir+"/"+a)
        shapes.append(res1.shape)
        res.append([res1,label])
    shapes = np.array(shapes)
    true_shapes = shapes>5000
    res = [a for a,b in zip(res,true_shapes) if b[0] == True]
    final_len = 16000
    final_dataset = [[resample_audio(a[0],final_len),a[1]] for a in res]
    return final_dataset

In [50]:
dataset = get_dataset(audio_dir)

In [51]:
import warnings
warnings.filterwarnings("ignore")
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

In [52]:

# feature_extractor = AutoFeatureExtractor.from_pretrained("sanchit-gandhi/whisper-medium-fleurs-lang-id")
# model = WhisperForAudioClassification.from_pretrained("sanchit-gandhi/whisper-medium-fleurs-lang-id")


In [53]:
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load the Whisper model in Hugging Face format:
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en").to(device)

In [54]:
emotion_label = {'anger' : 1, 'disgust' : 2, 'fear' : 3,
                 'joy' : 4, 'neutral' : 5, 'sadness' : 6, 'surprise' : 0}

In [55]:
encoder = model.model.encoder

In [57]:
final_dataset = []
for a in dataset:
    inputs = processor(a[0], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    with t.no_grad():
        outputs = encoder(inputs).last_hidden_state
        final_dataset.append([outputs.squeeze(0),emotion_label[a[1]]])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 for a in dataset:                                                                            │
│   3 │   inputs = processor(a[0], sampling_rate=16000, return_tensors="pt").input_features.to     │
│   4 │   with t.no_grad():                                                                        │
│ ❱ 5 │   │   outputs = encoder(inputs).last_hidden_state                                          │
│   6 │   │   final_dataset.append([outputs.squeeze(0),emotion_label[a[1]]])                       │
│   7                                                                                              │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/torch/nn/modules/module.py:889 in  │
│ _call_impl                                                                                       │
│                                                                                                  │
│    886 │   │   if torch._C._get_tracing_state():                                                 │
│    887 │   │   │   result = self._slow_forward(*input, **kwargs)                                 │
│    888 │   │   else:                                                                             │
│ ❱  889 │   │   │   result = self.forward(*input, **kwargs)                                       │
│    890 │   │   for hook in itertools.chain(                                                      │
│    891 │   │   │   │   _global_forward_hooks.values(),                                           │
│    892 │   │   │   │   self._forward_hooks.values()):                                            │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/transformers/models/whisper/modeli │
│ ng_whisper.py:858 in forward                                                                     │
│                                                                                                  │
│    855 │   │   │   │   │   │   (head_mask[idx] if head_mask is not None else None),              │
│    856 │   │   │   │   │   )                                                                     │
│    857 │   │   │   │   else:                                                                     │
│ ❱  858 │   │   │   │   │   layer_outputs = encoder_layer(                                        │
│    859 │   │   │   │   │   │   hidden_states,                                                    │
│    860 │   │   │   │   │   │   None,                                                             │
│    861 │   │   │   │   │   │   layer_head_mask=(head_mask[idx] if head_mask is not None else No  │
│                                                                                                  │
│ /raid/amana/miniconda3/envs/lamol/lib/python3.8/site-packages/torch/nn/modules/module.py:889 in  │
│ _call_impl                                                                                       │
│                                                                                                  │
│    886 │   │   if torch._C._get_tracing_state():                                                 │
│    887 │   │   │   result = self._slow_forward(*input, **kwargs)                                 │
│    888 │   │   else:                                                                             │
│ ❱  889 │   │   │   result = self.forward(*input, **kwargs)                                       │
│    890 │   │   for hook in itertools.chain(                

In [34]:
inputs = getVggoud_proc(audio_dir+"dia0_utt0.wav")

In [36]:
input_features = processor(
    inputs, sampling_rate=16000, return_tensors="pt"
).input_features

2024-01-04 16:18:10.653694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-04 16:18:13.973733: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [43]:
res = encoder(input_features.to(device))

In [47]:
res.last_hidden_state.shape

torch.Size([1, 1500, 384])